In [ ]:
!pip install pillow pycryptodome



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 5.6 MB/s eta 0:00:00


In [ ]:
from PIL import Image
import numpy as np
from Crypto.Cipher import AES
import os
from google.colab import files



In [ ]:


def pad_message(message):
    """Pads the message to a multiple of 16 bytes (AES block size)."""
    return message + (16 - len(message) % 16) * " "

def encrypt_message(message, key):
    """Encrypts the message using AES and encodes it in Base64."""
    key = key.ljust(16)[:16]  # Ensure key is 16 bytes
    cipher = AES.new(key.encode(), AES.MODE_ECB)
    encrypted_bytes = cipher.encrypt(pad_message(message).encode())
    return base64.b64encode(encrypted_bytes).decode()  # Convert to Base64

def decrypt_message(encrypted_message, key):
    """Decrypts an AES-encrypted Base64 encoded message."""
    key = key.ljust(16)[:16]
    cipher = AES.new(key.encode(), AES.MODE_ECB)
    decrypted_bytes = cipher.decrypt(base64.b64decode(encrypted_message))
    return decrypted_bytes.decode().strip()


In [ ]:

def embed_data(image_path, message, key, output_image):
    """Embeds an encrypted message into an image using LSB steganography."""
    img = Image.open(image_path)
    img = img.convert("RGB")
    pixels = np.array(img)

    # Encrypt the message & encode it in Base64
    encrypted_message = encrypt_message(message, key)
    binary_data = ''.join(format(ord(char), '08b') for char in encrypted_message) + '1111111111111110'  # End marker

    data_index = 0
    h, w, _ = pixels.shape
    np.random.seed(42)  # Ensure reproducibility
    pixel_indices = np.random.permutation(h * w)  # Shuffle pixels

    for index in pixel_indices:
        y, x = divmod(index, w)
        r, g, b = pixels[y, x]

        if data_index < len(binary_data):
            r = (r & 0xFE) | int(binary_data[data_index])
            data_index += 1
        if data_index < len(binary_data):
            g = (g & 0xFE) | int(binary_data[data_index])
            data_index += 1
        if data_index < len(binary_data):
            b = (b & 0xFE) | int(binary_data[data_index])
            data_index += 1

        pixels[y, x] = [r, g, b]

        if data_index >= len(binary_data):
            break

    stego_img = Image.fromarray(pixels)
    stego_img.save(output_image)
    print(f"Data embedded successfully! Saved as {output_image}")




In [ ]:
import base64
from PIL import Image
import numpy as np

def extract_data(image_path, key):
    """Extracts and decrypts the hidden message from an image with strict Base64 validation."""
    img = Image.open(image_path)
    img = img.convert("RGB")
    pixels = np.array(img)

    binary_data = ""
    h, w, _ = pixels.shape
    np.random.seed(42)  # Ensure the same randomized pattern is used
    pixel_indices = np.random.permutation(h * w)

    for index in pixel_indices:
        y, x = divmod(index, w)
        r, g, b = pixels[y, x]

        binary_data += str(r & 1)
        binary_data += str(g & 1)
        binary_data += str(b & 1)

    # Convert binary to text
    binary_data = [binary_data[i:i+8] for i in range(0, len(binary_data), 8)]
    extracted_message = ""

    for byte in binary_data:
        if byte == '11111110':  # Stop at end marker
            break
        extracted_message += chr(int(byte, 2))

    # 🔹 Strip unwanted characters (only keep Base64)
    extracted_message = ''.join(c for c in extracted_message if c in "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/=")

    # 🔹 Ensure extracted text is valid Base64 before decryption
    try:
        # Validate Base64 string
        base64.b64decode(extracted_message, validate=True)

        # If valid, decrypt it
        decrypted_message = decrypt_message(extracted_message, key)
        return decrypted_message
    except Exception as e:
        return f"Decryption Error: Invalid Base64 Extracted! {e}"


In [ ]:
from google.colab import files
uploaded = files.upload()
image_filename = list(uploaded.keys())[0]


Saving AICTE.jpeg to AICTE.jpeg


In [ ]:
import base64
output_image = "stego_image.png"
embed_data(image_filename, "Hello, Secure Steganography!", "securekey123", output_image)
files.download(output_image)


Data embedded successfully! Saved as stego_image.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
hidden_message = extract_data("stego_image.png", "securekey123")
print("Hidden Message:", hidden_message)



Hidden Message: Hello, Secure Steganography!
